In [88]:
from env import BSEnv
from env import Agent
import random

class SimpleAgent(Agent):
    def __init__(self, my_index, num_players):
        self.num_players= num_players


    def get_card_count(self, target_card, hand):
        num_target_card = 0
        for card in hand:
            if card == target_card:
                num_target_card+=1
        return num_target_card

    def get_card(self, intended_card, hand):
        # check if has card and if so, plays it
        print(hand)
        card_count = self.get_card_count(intended_card, hand)
        if card_count > 0:
            return intended_card, card_count
        # play a random card if not
        return hand[0], 1

    def get_call_bs(self, player_index, card, card_amt, hand):
        # calls BS if knows it is BS
        card_count = self.get_card_count(card, hand)
        if card_count + card_amt > 4:
            return True
        # random BS call for fun
        if random.random() < 0.1:
            return True

        return False

    def give_info(self, player_indexes_picked_up):
        pass

In [89]:
env = BSEnv(agent_types = [SimpleAgent, SimpleAgent, SimpleAgent,SimpleAgent])

In [90]:
env.reset()
env.run_game()

-----------
Turn: 0 Card to Play: A
Pile []
['8', '2', '7', 'J', '9', '8', '6', '10', 'K', '9', '6', '4', '2']
Player 0 current hand ['8', '2', '7', 'J', '9', '8', '6', '10', 'K', '9', '6', '4', '2']
Player 0 plays 8 1 time(s).
Player 0 new hand ['2', '7', 'J', '9', '8', '6', '10', 'K', '9', '6', '4', '2']
-----------
Turn: 1 Card to Play: 2
Pile ['8']
['4', '7', '6', 'K', '9', '10', '10', '5', '5', 'Q', '2', '5', '9']
Player 1 current hand ['4', '7', '6', 'K', '9', '10', '10', '5', '5', 'Q', '2', '5', '9']
Player 1 plays 2 1 time(s).
Player 1 new hand ['4', '7', '6', 'K', '9', '10', '10', '5', '5', 'Q', '5', '9']
-----------
Turn: 2 Card to Play: 3
Pile ['8', '2']
['7', 'A', '3', '3', '8', 'A', 'A', '5', '2', 'J', '4', '4', 'Q']
Player 2 current hand ['7', 'A', '3', '3', '8', 'A', 'A', '5', '2', 'J', '4', '4', 'Q']
Player 2 plays 3 2 time(s).
Player 2 new hand ['7', 'A', '8', 'A', 'A', '5', '2', 'J', '4', '4', 'Q']
-----------
Turn: 3 Card to Play: 4
Pile ['8', '2', '3', '3']
['A', '8